<a href="https://colab.research.google.com/github/trongtn2110/CS114.L21/blob/master/Colab/Sarcasm_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**CUNG CẤP CÁC THƯ VIỆN CẦN THIẾT**

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
from keras.preprocessing import text, sequence
from sklearn.model_selection import train_test_split
from string import punctuation
import keras
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional,GRU


#**LẤY DỮ LIỆU ĐÃ THU THẬP**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/Data/data_sarcasm/merged_new_42.json',orient='records')
df.head()

#**KHÁM PHÁ DỮ LIỆU**

In [ ]:
df.isna().sum() # Kiểm tra NaN

In [ ]:
sns.set_style("dark")
sns.countplot(df.is_sarcastic)

#**XỬ LÝ DATA**

##**LOẠI BỎ CÁC STOPWORDS**


> Stopwords là những từ tiếng Anh không bổ sung nhiều ý nghĩa cho một câu. Chúng có thể được bỏ qua một cách an toàn mà không làm mất đi ý nghĩa của câu. Ví dụ, những từ như, anh ấy, có, vv Những từ như vậy đã được ghi lại điều này trong ngữ liệu có tên là ngữ liệu.


In [ ]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

##**XÓA CÁC KÍ TỰ KHÔNG THỂ XỬ LÝ**

In [ ]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Xóa các dấu ngoặc
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)
# Xóa URL's
def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)
#Xóa các stopwords
def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop:
            final_text.append(i.strip())
    return " ".join(final_text)
def denoise_text(text):
    text = text.lower()
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text
df['headline']=df['headline'].apply(denoise_text)

##**CHUYỂN ĐỔI DATA CHO PHÙ HỢP VỚI MODEL WORD2VEC**

In [ ]:
words = []
for i in df.headline.values:
    words.append(i.split())
words[:5]

In [ ]:
import gensim
EMBEDDING_DIM = 200
#Tạo model w2v
w2v_model = gensim.models.Word2Vec(sentences = words , size=EMBEDDING_DIM , min_count = 1)

##**MÃ HÓA DATA**

In [ ]:
tokenizer = text.Tokenizer()
tokenizer.fit_on_texts(words)
tokenized_train = tokenizer.texts_to_sequences(words)
x = sequence.pad_sequences(tokenized_train, maxlen = 20)

In [ ]:
# print(tokenizer.word_counts)
# print(tokenizer.document_count)
# print(tokenizer.word_index)
# print(tokenizer.word_docs)

In [ ]:
#Cộng 1 vì dành cho các từ không biết, vector này sẽ toàn 0
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
# Function to create weight matrix from word2vec gensim model
def get_weight_matrix(model, vocab):
    # total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    weight_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
    for word, i in vocab.items():
        weight_matrix[i] = model[word]
    return weight_matrix

In [ ]:
embedding_vectors = get_weight_matrix(w2v_model, tokenizer.word_index)

#**TRAIN MODEL**

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, output_dim=EMBEDDING_DIM, weights=[embedding_vectors], input_length=20, trainable=True))
#LSTM
model.add(Bidirectional(LSTM(units=128 , recurrent_dropout = 0.3 , dropout = 0.3,return_sequences = True)))
model.add(Bidirectional(GRU(units=32 , recurrent_dropout = 0.1 , dropout = 0.1)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer=keras.optimizers.Adam(lr = 0.01), loss='binary_crossentropy', metrics=['acc'])

del embedding_vectors

In [ ]:
model.summary()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, df.is_sarcastic , test_size = 0.3 , random_state = 0) 

In [ ]:
history = model.fit(x_train, y_train, batch_size = 128 , validation_data = (x_test,y_test) , epochs = 3)

In [ ]:
print("Accuracy of the model on Training Data is - " , model.evaluate(x_train,y_train)[1]*100)
print("Accuracy of the model on Testing Data is - " , model.evaluate(x_test,y_test)[1]*100)

#**NHẬN XÉT**
- Thấy kết quả accuracy score trên Testing Data là 92.4 là khá cao so với accuracy score trên Training Data là 99.9